In [8]:
import pandas as pd
import numpy as np
geneToLength = {}
genes = pd.read_csv('/data/khandekara2/bed_CpGs/mart_export.bed', sep='\t', header=None)
for gene, length in zip(genes.iloc[:,5], genes.iloc[:,3]):
    geneToLength[gene] = length

In [9]:
print (geneToLength['ACBD7'])

1322


In [2]:
#using the MALY cancer data and it's corresponding benign tissue data, 
#find those genes whose methylation levels in coding regions departed significantly(p <= 0.05) from the cds average 
import pandas as pd
import numpy as np
df = pd.read_csv('P_cg_results_highly_methylated.tsv', sep='\t')#normal tissue
df2 = pd.read_csv('P_cg_results_lowly_methylated.tsv', sep='\t')#normal tissue
maly = pd.read_csv('P_cg_results_MALY_highly_methylated.tsv', sep='\t')#tumor
maly2 = pd.read_csv('P_cg_results_MALY_lowly_methylated.tsv', sep='\t')#tumor
thymus = df[df['tissue'] == 'Thymus']
thymus2 = df2[df2['tissue'] == 'Thymus']
thymus = thymus.dropna()
thymus2 = thymus2.dropna()
thymus2 = thymus2.filter(['p_value'], axis=1)
maly = maly.dropna()
maly2 = maly2.dropna()
maly2 = maly2.filter(['p_value'], axis=1)
thymus2.rename(columns={'p_value': 'p_value_low'}, inplace=True)
maly2.rename(columns={'p_value': 'p_value_low'}, inplace=True)
maly['p_value_low'] = maly2['p_value_low']
thymus['p_value_low'] = thymus2['p_value_low']
thymus['high_benign'] = np.where(thymus['p_value'] <= 0.05, 'yes', 'no')
thymus['baseline_benign'] = np.where((thymus['p_value'] > 0.05) & (thymus['p_value_low'] > 0.05), 'yes', 'no')
thymus['low_benign'] = np.where(thymus['p_value_low'] <= 0.05, 'yes', 'no')
maly['low'] = np.where(maly['p_value_low'] <= 0.05, 'yes', 'no')
maly['baseline'] = np.where((maly['p_value'] > 0.05) & (maly['p_value_low'] > 0.05), 'yes', 'no')
maly['high'] = np.where(maly['p_value'] <= 0.05, 'yes', 'no')

In [3]:
maly.head()

,tissue,gene,total,methylated,p_value,prior,length,p_value_low,low,baseline,high
0,Thymus,CCDC24,31,14,0.999309,0.71,1463,0.002283,yes,no,no
2,Thymus,C1orf50,21,4,1.000000,0.71,996,0.000001,yes,no,no
3,Thymus,PPFIA4,79,55,0.658577,0.71,5045,0.435182,no,yes,no
4,Thymus,CAP1,24,20,0.132087,0.71,3105,0.947553,no,yes,no
5,Thymus,MMACHC,27,21,0.294013,0.71,2981,0.838458,no,yes,no


In [4]:
#print statistics for benign tissue(thymus)...how many genes were in each category
high_benign = [e for e in thymus['high_benign'] if e == 'yes']
low_benign = [e for e in thymus['low_benign'] if e == 'yes']
baseline_benign = [e for e in thymus['baseline_benign'] if e == 'yes']
print (len(high_benign))
print (len(low_benign))
print (len(baseline_benign))





8856
4470
4898


In [5]:
#print statistics for cancer samples(MALY)...how many genes were in each category
high_maly = [e for e in maly['high'] if e == 'yes']
low_maly = [e for e in maly['low'] if e == 'yes']
baseline_maly = [e for e in maly['baseline'] if e == 'yes']
print (len(high_maly))
print (len(low_maly))
print (len(baseline_maly))

6329
4680
6939


In [6]:
#merge benign and cancer dataframes on gene name so that we have all information for a gene in one table and write out to tsv file
maly = maly.drop('tissue', axis=1)
merged = pd.merge(thymus, maly, on='gene', how='inner')
merged['p_value_y'] = merged['p_value_y'].apply(lambda x: round(x, 2))
merged.head()
merged.to_csv('maly_cds_differential_methylation.tsv', index=False, sep='\t')

In [7]:
#now conduct cds differential methylation analysis
merged.head()




,tissue,gene,total_x,methylated_x,p_value_x,prior_x,length_x,p_value_low_x,high_benign,baseline_benign,low_benign,total_y,methylated_y,p_value_y,prior_y,length_y,p_value_low_y,low,baseline,high
0,Thymus,A1BG,130,83,1.0,0.74,1722,0.01,no,no,yes,108,76,0.60,0.71,1722,0.478854,no,yes,no
1,Thymus,A1CF,28,27,0.0,0.74,9221,1.00,yes,no,no,24,22,0.02,0.71,9221,0.997091,no,no,yes
2,Thymus,A2ML1,78,74,0.0,0.74,5274,1.00,yes,no,no,72,62,0.00,0.71,5274,0.999221,no,no,yes
3,Thymus,A2M,78,78,0.0,0.74,4844,1.00,yes,no,no,58,50,0.01,0.71,4844,0.998054,no,no,yes
4,Thymus,A3GALT2,99,17,1.0,0.74,1023,0.00,no,no,yes,84,62,0.33,0.71,1023,0.751231,no,yes,no


In [8]:
#filter for genes that have at least x CpG's in them
m = merged[(merged['total_x'] >= 10) & (merged['total_y'] >= 10)]
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15073 entries, 0 to 17881
Data columns (total 20 columns):
tissue             15073 non-null object
gene               15073 non-null object
total_x            15073 non-null int64
methylated_x       15073 non-null int64
p_value_x          15073 non-null float64
prior_x            15073 non-null float64
length_x           15073 non-null int64
p_value_low_x      15073 non-null float64
high_benign        15073 non-null object
baseline_benign    15073 non-null object
low_benign         15073 non-null object
total_y            15073 non-null int64
methylated_y       15073 non-null int64
p_value_y          15073 non-null float64
prior_y            15073 non-null float64
length_y           15073 non-null int64
p_value_low_y      15073 non-null float64
low                15073 non-null object
baseline           15073 non-null object
high               15073 non-null object
dtypes: float64(6), int64(6), object(8)
memory usage: 2.4+ MB


In [9]:
#9 possible transitions
baseToBase = set([])
baseToHigh = set([])
baseToLow = set([])

lowToHigh = set([])
lowToBase = set([])
lowToLow = set([])

highToLow = set([])
highToBase = set([])
highToHigh = set([])



In [10]:
for index, row in m.iterrows():
    if row['baseline_benign'] == 'yes' and row['baseline'] == 'yes':
        baseToBase.add(row['gene'])
    elif row['baseline_benign'] == 'yes' and row['high'] == 'yes':
        baseToHigh.add(row['gene'])
    elif row['baseline_benign'] == 'yes' and row['low'] == 'yes':
        baseToLow.add(row['gene'])
    elif row['low_benign'] == 'yes' and row['high'] == 'yes':
        lowToHigh.add(row['gene'])
    elif row['low_benign'] == 'yes' and row['baseline'] == 'yes':
        lowToBase.add(row['gene']) 
    elif row['low_benign'] == 'yes' and row['low'] == 'yes':
        lowToLow.add(row['gene'])
    elif row['high_benign'] == 'yes' and row['low'] == 'yes':
        highToLow.add(row['gene'])
    elif row['high_benign'] == 'yes' and row['baseline'] == 'yes':
        highToBase.add(row['gene'])
    elif row['high_benign'] == 'yes' and row['high'] == 'yes':
        highToHigh.add(row['gene'])
    else:
        print(row)
    
        

In [11]:
print (len(baseToBase))
print (len(baseToHigh))
print (len(baseToLow))

print (len(lowToHigh))
print (len(lowToBase))
print (len(lowToLow))

print (len(highToLow))
print (len(highToBase))
print (len(highToHigh))

2130
289
526
112
392
3717
98
1997
5812


In [ ]:
oncogenes = m[(m['baseline_benign'] == 'yes') & (m['high_benign_y'] == 'yes')]

In [ ]:
oncogenes.head()

In [ ]:
import csv
with open('lowToHigh.txt', 'w') as csvout:
    writer = csv.writer(csvout, delimiter='\n')
    writer.writerow(list(lowToHigh))
    

In [ ]:
# lowToHigh_df = m[m['gene'].isin(lowToHigh)]
lowToHigh_df.head()

In [13]:
census = pd.read_csv('/Users/khandekara2/Documents/methylationProject/01_data/Census_all_2018.csv')
census.head()

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974,10:50806729-50859940,2,NaN,10q11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ENSG00000148584,Q9NQ94"
1,ABI1,abl-interactor 1,10006,10:26748570-26860863,1,Yes,10p11.2,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI1,ENSG00000136754,Q8IZP0,SSH3BP1"
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25,9:130835447-130885683,1,Yes,9q34.1,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007,P00519"
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27,1:179107718-179143044,1,NaN,1q24-q25,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ENSG00000143322,P42684"
4,ACKR3,atypical chemokine receptor 3,57007,2:236580466-236581554,1,Yes,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"A8K6J4,ACKR3,CMKOR1,CXCR7,ENSG00000144476,P251..."


In [16]:
cancer_genes = list(census['Gene Symbol'])
baseToBase_cancer = set([])
baseToHigh_cancer = set([])
baseToLow_cancer = set([])

lowToHigh_cancer = set([])
lowToBase_cancer = set([])
lowToLow_cancer = set([])

highToLow_cancer = set([])
highToBase_cancer = set([])
highToHigh_cancer = set([])


In [17]:
for index, row in m.iterrows():
    if row['baseline_benign'] == 'yes' and row['baseline'] == 'yes' and row['gene'] in cancer_genes:
        baseToBase_cancer.add(row['gene'])
    elif row['baseline_benign'] == 'yes' and row['high'] == 'yes' and row['gene'] in cancer_genes:
        baseToHigh_cancer.add(row['gene'])
    elif row['baseline_benign'] == 'yes' and row['low'] == 'yes' and row['gene'] in cancer_genes:
        baseToLow_cancer.add(row['gene'])
    elif row['low_benign'] == 'yes' and row['high'] == 'yes' and row['gene'] in cancer_genes:
        lowToHigh_cancer.add(row['gene'])
    elif row['low_benign'] == 'yes' and row['baseline'] == 'yes' and row['gene'] in cancer_genes:
        lowToBase_cancer.add(row['gene']) 
    elif row['low_benign'] == 'yes' and row['low'] == 'yes' and row['gene'] in cancer_genes:
        lowToLow_cancer.add(row['gene'])
    elif row['high_benign'] == 'yes' and row['low'] == 'yes' and row['gene'] in cancer_genes:
        highToLow_cancer.add(row['gene'])
    elif row['high_benign'] == 'yes' and row['baseline'] == 'yes' and row['gene'] in cancer_genes:
        highToBase_cancer.add(row['gene'])
    elif row['high_benign'] == 'yes' and row['high'] == 'yes' and row['gene'] in cancer_genes:
        highToHighbaseToBase = set([])
    else:
        pass

In [18]:
print (len(baseToBase_cancer))
print (len(baseToHigh_cancer))
print (len(baseToLow_cancer))

print (len(lowToHigh_cancer))
print (len(lowToBase_cancer))
print (len(lowToLow_cancer))

print (len(highToLow_cancer))
print (len(highToBase_cancer))
print (len(highToHigh_cancer))

70
9
28
2
17
122
7
63
0


In [19]:
print (lowToHigh_cancer)

{'NR4A3', 'CTNND2'}


In [22]:
lowToHigh_cancer_df = m[m['gene'].isin(lowToHigh_cancer)]
lowToHigh_cancer_df

,tissue,gene,total_x,methylated_x,intermediate_x,p_value_x,prior_x,length_x,p_value_low_x,high_benign,...,total_y,methylated_y,intermediate_y,p_value_y,prior_y,length_y,p_value_low_y,low,baseline,high
1423,Thymus,PAX7,72,36,19,1.00,0.72,2260,0.00,no,...,29,24,73,0.03,0.65,2260,0.989859,no,no,yes
1688,Thymus,PRRX1,40,19,3,1.00,0.72,2217,0.00,no,...,11,11,37,0.01,0.65,2217,1.000000,no,no,yes
4058,Thymus,TBX3,102,19,63,1.00,0.72,4723,0.00,no,...,54,52,138,0.00,0.65,4723,1.000000,no,no,yes
9588,Thymus,ALK,164,105,15,0.99,0.72,6220,0.02,no,...,58,47,154,0.01,0.65,6220,0.997658,no,no,yes
10000,Thymus,CTNNA2,211,100,4,1.00,0.72,4349,0.00,no,...,69,68,202,0.00,0.65,4349,1.000000,no,no,yes
12316,Thymus,WWTR1,39,17,20,1.00,0.72,5030,0.00,no,...,11,11,55,0.01,0.65,5030,1.000000,no,no,yes
12778,Thymus,FAT4,287,174,10,1.00,0.72,16123,0.00,no,...,71,63,231,0.00,0.65,16123,0.999999,no,no,yes
13492,Thymus,CTNND2,196,118,10,1.00,0.72,5481,0.00,no,...,90,83,159,0.00,0.65,5481,1.000000,no,no,yes
15878,Thymus,FKBP9,66,33,6,1.00,0.72,3462,0.00,no,...,28,24,63,0.01,0.65,3462,0.996320,no,no,yes
17111,Thymus,NR4A3,122,18,66,1.00,0.72,2588,0.00,no,...,14,14,90,0.00,0.65,2588,1.000000,no,no,yes


In [ ]:
{'WWTR1', 'PAX7', 'NR4A3', 'FKBP9', 'CTNNA2', 'PRRX1', 'FAT4', 'CTNND2', 'TBX3', 'ALK'}